# Test script of SomaNews Clustering
By Datetime : 2016-08-29 ~ 2016-09-05

In [1]:
import numpy as np
import pandas as pd

## Load Data
Load Data from database

In [2]:
train = pd.read_pickle("../datastore/train.p")
train = train.drop(['_id', 'author', 'description', 'link', 'imageURL', 'providerNewsID'], axis=1)
train.head()

,category,content,provider,publishedAt,title
0,사회 > 카드뉴스,과연 진실은... \n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] \n\n[ 조선일...,chosun,2016-09-21 19:28:00,[카드뉴스] 대통령은 왜 흙을 밟지 않았을까요?
1,사회 > 카드뉴스,'세기의 연인'이라 불렸던 커플\n\n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] ...,chosun,2016-09-21 19:10:00,[카드뉴스] '브란젤리나'의 시작과 끝
2,사회 > 카드뉴스,당신의 목소리는 그 누구의 목소리보다 귀하게 쓰였습니다. 기사보기 : https:...,chosun,2016-09-21 19:03:00,[카드뉴스] 목소리를 가장 귀하게 쓴 한 성우 지망생
3,문화 > 오늘의 운세,36년생 정도를 지키면 무난한 날. 48년생 결정할 일 결정하도록. 60년생 가뭄에...,chosun,2016-09-21 18:00:00,[오늘의 운세] 9월 22일 목요일(음력 8월 22일 丁未)
4,스포츠ㆍ연예 > 스포츠 > 종합,,chosun,2016-09-21 03:00:00,[오늘의 경기] 2016년 9월 21일


In [3]:
train = train[pd.to_datetime(train.publishedAt).dt.year == 2016]
train.publishedAt.head()

0   2016-09-21 19:28:00
1   2016-09-21 19:10:00
2   2016-09-21 19:03:00
3   2016-09-21 18:00:00
4   2016-09-21 03:00:00
Name: publishedAt, dtype: datetime64[ns]

## Preprocessing
0. Datetime (16-09-11 ~ 16-09-17)
1. Remove stopwords (regex, hanja)
2. POS Tagging with KoNLPy, Mecab

In [4]:
import datetime
from konlpy.tag import Mecab
import hanja
import re

In [5]:
train["newsNum"] = 1

In [6]:
train.set_index(train['publishedAt'], inplace=True)
train.head()

,category,content,provider,publishedAt,title,newsNum
publishedAt,,,,,,
2016-09-21 19:28:00,사회 > 카드뉴스,과연 진실은... \n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] \n\n[ 조선일...,chosun,2016-09-21 19:28:00,[카드뉴스] 대통령은 왜 흙을 밟지 않았을까요?,1
2016-09-21 19:10:00,사회 > 카드뉴스,'세기의 연인'이라 불렸던 커플\n\n[ 조선일보 카드뉴스가 더 보고 싶다면 ?] ...,chosun,2016-09-21 19:10:00,[카드뉴스] '브란젤리나'의 시작과 끝,1
2016-09-21 19:03:00,사회 > 카드뉴스,당신의 목소리는 그 누구의 목소리보다 귀하게 쓰였습니다. 기사보기 : https:...,chosun,2016-09-21 19:03:00,[카드뉴스] 목소리를 가장 귀하게 쓴 한 성우 지망생,1
2016-09-21 18:00:00,문화 > 오늘의 운세,36년생 정도를 지키면 무난한 날. 48년생 결정할 일 결정하도록. 60년생 가뭄에...,chosun,2016-09-21 18:00:00,[오늘의 운세] 9월 22일 목요일(음력 8월 22일 丁未),1
2016-09-21 03:00:00,스포츠ㆍ연예 > 스포츠 > 종합,,chosun,2016-09-21 03:00:00,[오늘의 경기] 2016년 9월 21일,1


In [7]:
weekly_df = train.resample('W-Mon').agg({'newsNum': np.sum}).fillna(0)
weekly_df

,newsNum
publishedAt,
2016-01-04,585
2016-01-11,1645
2016-01-18,1847
2016-01-25,2757
2016-02-01,2830
2016-02-08,2298
2016-02-15,1972
2016-02-22,2769
2016-02-29,2941


In [8]:
dstart = datetime.date(2016,8,29)
dend = dstart + datetime.timedelta(weeks=1)
dend

datetime.date(2016, 9, 5)

In [9]:
train = train[(train["publishedAt"] > dstart) & (train["publishedAt"] < dend)]
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3068 entries, 2016-09-04 23:28:00 to 2016-08-29 00:51:00
Data columns (total 6 columns):
category       3068 non-null object
content        3068 non-null object
provider       3068 non-null object
publishedAt    3068 non-null datetime64[ns]
title          3068 non-null object
newsNum        3068 non-null int64
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 167.8+ KB


In [10]:
mecab = Mecab()
print(mecab.nouns(u'네, 안녕하세요'))

['안녕']


In [11]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', ',', text)
    text = text.replace("카드뉴스", '')
    return text

In [12]:
def tokenize(data):
    return [' '.join(str(e) for e in mecab.nouns(data))]

In [13]:
train['title'] = train['title'].apply(lambda text: text_cleaning(text)).astype('U')
title = [tokenize(each[1]['title']) for each in train.iterrows()]
title

[['전기 저장 년 뒤 억 달러 시장'],
 ['할인 요금제 기간 년'],
 ['중국 발 공급 과잉 저유 먹구름 정유 업계 긴장'],
 ['레고 로봇 스마트 캐디 게임 칫솔 독서 기업'],
 ['경쟁사 핵심 부품 건 기술력 자신 때문'],
 ['올해 내 미 사물 인터넷 서비스 단계 타국 확대'],
 ['영화 예고편 줄'],
 ['영화 예고편 줄'],
 ['원자재 값 급등 중규 국내 배 터리사 중고'],
 ['원자재 값 급등 중규 국내 배 터리사 중고'],
 ['파업 중 한국 사장 직원 이메일'],
 ['면세점 덩치 한국 신규 곳 적자'],
 ['면세점 덩치 한국 신규 곳 적자'],
 ['보장 보험료 올해 최대'],
 ['교실 밖 아이 선생 질문'],
 ['배달 앱 프랑스 요리 부심'],
 ['오늘 운세 월 일 월요일 음력 월 일 경인'],
 ['미래 첨단 무기 웨어 러블 로봇 주요 기술 확보'],
 ['차세대 무기 체계 레이저 무기 투자 집중'],
 ['국내 최대 규모 지상군 전시회'],
 ['천 년 고도 경주 사극 세트 장'],
 ['소년 세 생일 축하 편지'],
 ['아가 네 세상'],
 ['다운 증후군 종업원 은퇴 파티'],
 ['투견 눈물'],
 ['사설 방어 무기 하나 달 소란 정국 마비 나라'],
 ['사설 이번 서경배 억 줄 기업 노블레스 오 블리 주'],
 ['사설 재벌 오너 이름 수십억 연봉'],
 ['만물상 퇴비 냄새'],
 ['박성현 공동 선두'],
 ['터치 코리아 당신 애정 취약 계층 이 십'],
 ['트렌드 돋보기 이화'],
 ['가슴 한시'],
 ['이미도 무비 식도'],
 ['조선 주말 하이라이트 모란봉 클럽 외'],
 ['조선 추석 준비 법'],
 ['조선 남자 인생'],
 ['윤성호 허정구 배 아마추어 골프 련패'],
 ['라운드 홀인원 개'],
 ['오늘 경기 년 월 일'],
 ['미 시 풋볼 선수 이번 돼지 양말 논란'],
 ['내일 경기 년 월 일'],
 ['스포츠 브리핑 강정호 재활 경기 홈런'],
 ['정신줄 수비 고질

## Training
Topic modeling : Latent Dirichlet allocation (LDA)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time

In [15]:
vectorizer = TfidfVectorizer(lowercase=False)
title_flat = [item for sublist in title for item in sublist]
x_list = vectorizer.fit_transform(title_flat)
x_list

<3068x5904 sparse matrix of type '<class 'numpy.float64'>'
	with 16204 stored elements in Compressed Sparse Row format>

### Topic Modeling
Latent Dirichlet Allocation (LDA)

In [16]:
from sklearn.decomposition import LatentDirichletAllocation

In [17]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [18]:
t0 = time()
lda = LatentDirichletAllocation(n_topics=15, max_iter=5, n_jobs=-1)
lda.fit(x_list)
print("Done in %0.3fs." % (time() - t0))

Done in 6.378s.


In [19]:
feature_names = vectorizer.get_feature_names()
print_top_words(lda, feature_names, 20)

Topic #0:
경제 송희영 시민 김진태 주필 친문 문제 의료 사랑 지표 인사 화제 현장 안희정 어린이 분양 그룹 공사 인생 대선
Topic #1:
인사 미래 조선일보 조선 병원 서울 대학 기업 시대 일본 예산 청와대 최고 발생 경영 뉴스 종합 위안부 단신 회장
Topic #2:
청년 청소년 공개 일자리 토크 연속 발견 자리 생명 비상 지카 위원 총리 무엇 인정 천억 담배 대신 아이 아시아
Topic #3:
전형 수시 전략 학생 대입 필승 가능 확대 지원 신설 선발 올해 영화 개월 인천 면접 적성 모집 공감 고사
Topic #4:
대학교 국회 추경 부산 청문회 금융 올림픽 운동 희망 비판 처리 음악 의장 정책 발언 민주 장인상 정의 정기 부고
Topic #5:
수능 여성 산업 최저 사진 사망 야당 누구 기준 출신 폭로 특별 적용 독립 기사 아침 캠프 부회장 이정현 충북
Topic #6:
소식 추석 단독 내년 트럼프 시장 동정 정치 정부 마음 역사 폭발 주택 공급 호세프 감동 공항 탄핵 선물 성장
Topic #7:
사회 공헌 검찰 구조 환자 국민 추진 수사 과학 피해자 콜레라 마을 교육 롯데 법원 이상 공부 국제 간염 조정
Topic #8:
병우 사드 부고 별세 운세 오늘 모친상 책임 언론 한가위 이석수 사표 반대 사고 누리 배치 이번 경찰 사설 남도
Topic #9:
포토 날씨 지역 나라 우리 정보 우수 정상 대통령 해외 회담 폭탄 불법 노동 사드 강화 오늘 방송 아기 테슬라
Topic #10:
야구 학교 프로 폭염 관리 한진 부동산 한국 조사 해운 현대 결정 국내 전국 자유 우승 역대 도입 하나 법정
Topic #11:
생각 세월 사업 전자 김재수 감사 경주 세트 인상 디자인 생활 김영훈 위원장 장관 장학금 기록 판매 유럽 임금 때문
Topic #12:
사람 해운 한진 한겨레 뉴스룸 노동자 사설 금리 관리 물류 그림판 중국 축구 의혹 증권 신문 법정 사과 마지막 피해
Topic #13:
경기 오늘 노트 여자 배터리 갤럭시 확정 삼성 삼성전자 채널 리콜 세계

## Result
1. Model Selection
2. Preprocessing
3. Feature Engineering
4. Parameter Tuning
5. Score Function